In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part One 
- not trackable for part 2, but first thought 

In [2]:
class CubeDirection:
    def __init__(self,line):
        light, dimensions = line.strip().split()
        
        if light == 'on':
            on = True
        else:
            on=False
        self.is_on = on
        
        dims = dimensions.split(',')
        self.x_limits = [int(i) for i in dims[0][2:].split('..')]
        self.y_limits = [int(i) for i in dims[1][2:].split('..')]
        self.z_limits = [int(i) for i in dims[2][2:].split('..')]
    
    def calculate_size(self,):
        x_length = self.x_limits[1] - self.x_limits[0]+1
        y_length = self.y_limits[1] - self.y_limits[0]+1
        z_length = self.z_limits[1] - self.z_limits[0]+1

        return x_length * y_length* z_length
    def give_internal_elements(self, part_one=True):
        def cap_fifty(element):
            return min(max(-50, element), 50)
        
        x_lims = self.x_limits
        y_lims = self.y_limits
        z_lims = self.z_limits
        
        elements = []
        if part_one:
            if x_lims[1] < -50 or y_lims[1] < -50 or z_lims[1] < -50:
                return elements
            if x_lims[0] > 50 or y_lims[0] >  50 or z_lims[0] > 50:
                return elements
            x_lims = [ cap_fifty(i) for i in x_lims ]
            y_lims = [ cap_fifty(i) for i in y_lims ]
            z_lims = [ cap_fifty(i) for i in z_lims ]
       
        for x in range(x_lims[0], x_lims[1]+1):
            for y in range(y_lims[0], y_lims[1]+1):
                for z in range(z_lims[0], z_lims[1]+1):
                    elements.append(f'{x},{y},{z}')
                    
        return elements

In [3]:
def read_directions(file_path):
    cube_directions =[]
    with open(file_path, 'r') as f:
        cube_directions = f.readlines()
        cube_directions = [ line.strip() for line in cube_directions]
        
    on_lights = set([])
    for line in cube_directions:
        cube = CubeDirection(line)
        if cube.is_on:
            on_lights = on_lights.union(cube.give_internal_elements())
        else:
            remove = cube.give_internal_elements()
            on_lights = on_lights - set(remove)
    
    
    return len(on_lights)
        

In [4]:
read_directions('data/test_22.txt')
#read_directions('data/test_22_1.txt')

590784

474140

In [5]:
%%time
read_directions('data/input_22.txt')

CPU times: user 5.63 s, sys: 565 ms, total: 6.2 s
Wall time: 6.2 s


609563

# Part two

In [64]:
class Cuboid:
    def __init__(self,
                 line=None,
                 on=True,
                 x_limits=None,
                 y_limits=None,
                 z_limits=None,
                ):
        if line is not None:
            light, dimensions = line.strip().split()
            
            if light == 'on':
                on = True
            else:
                on=False
            self.is_on = on
            
            dims = dimensions.split(',')
            self.x_limits = [int(i) for i in dims[0][2:].split('..')]
            self.y_limits = [int(i) for i in dims[1][2:].split('..')]
            self.z_limits = [int(i) for i in dims[2][2:].split('..')]
            
        else:
            self.is_on = on
            self.x_limits = x_limits
            self.y_limits = y_limits
            self.z_limits = z_limits
            
        self.size = self.calculate_size()
        
    def __str__(self):
        dims = [self.x_limits,
                self.y_limits,
                self.z_limits]
        return ',  '.join([ ' to '.join([str(p) for p in d]) for d in dims]) 
    
    def calculate_size(self):
        x_length = self.x_limits[1] - self.x_limits[0]+1
        y_length = self.y_limits[1] - self.y_limits[0]+1
        z_length = self.z_limits[1] - self.z_limits[0]+1

        return x_length * y_length* z_length
    
    def check_disjoint(self, other):
        x_out = (    (self.x_limits[0] > other.x_limits[1])
                  or (other.x_limits[0] > self.x_limits[1])
                )
        y_out = (    (self.y_limits[0] > other.y_limits[1])
                  or (other.y_limits[0] > self.y_limits[1])
                )
        z_out = (    (self.z_limits[0] > other.z_limits[1])
                  or (other.z_limits[0] > self.z_limits[1])
                )
        return (x_out or y_out or z_out)
    
    def get_dimension_interction(self, s_dim, o_dim):
        return [max(s_dim[0], o_dim[0]), min(s_dim[1], o_dim[1]) ]

        
    def __add__(self, other):
        disjoint = self.check_disjoint(other)
        if disjoint:
            return None
        
        return Cuboid(
                       x_limits=self.get_dimension_interction(self.x_limits,other.x_limits),
                       y_limits=self.get_dimension_interction(self.y_limits,other.y_limits),
                       z_limits=self.get_dimension_interction(self.z_limits,other.z_limits),
                    )
        
    def exclude_region(self, dim, exclusion):
        ranges = []
        if dim[0] < exclusion[0]:
            ranges.append( [ dim[0], min(exclusion[0]-1, dim[1]) ])
        if dim[1] > exclusion[1]:
            ranges.append( [ max(exclusion[1]+1,dim[0]), dim[1] ])
        return ranges
    
    def __sub__(self, other):
        disjoint = self.check_disjoint(other)
        if disjoint:
            return [self]
        
        intersection = self+other
        
        cuboids=[]
        for xlims in self.exclude_region(self.x_limits, intersection.x_limits):
            cuboids.append(
                Cuboid(
                        x_limits=xlims,
                        y_limits=self.y_limits,
                        z_limits=self.z_limits,
                    )
            )
            
        for ylims in self.exclude_region(self.y_limits, intersection.y_limits):
            cuboids.append(
                Cuboid(
                        x_limits=intersection.x_limits,
                        y_limits=ylims,
                        z_limits=self.z_limits,
                    )
            )
        for zlims in self.exclude_region(self.z_limits, intersection.z_limits):
            cuboids.append(
                Cuboid(
                        x_limits=intersection.x_limits,
                        y_limits=intersection.y_limits,
                        z_limits=zlims,
                    )
            )
        return cuboids


In [65]:
def read_directions_part2(file_path):
    cube_directions =[]
    with open(file_path, 'r') as f:
        cube_directions = f.readlines()
        cube_directions = [ line.strip() for line in cube_directions]
        
    cuboids = [Cuboid(line=cube_directions[0])]
    
    for i,line in enumerate(cube_directions[1:]):
        new_cuboid = Cuboid(line=line)
        
        next_cuboids = []
        for cube in cuboids:
            next_cuboids.extend( cube-new_cuboid )
                    
        if new_cuboid.is_on:
            next_cuboids.append(new_cuboid)
                    
        cuboids = next_cuboids.copy()
    
    total=0
    for cube in cuboids:
        total += cube.size
    
    return total
        

In [66]:
%%time
read_directions_part2('data/test_22_1.txt')

CPU times: user 27.5 ms, sys: 1.75 ms, total: 29.2 ms
Wall time: 29.1 ms


2758514936282235

In [67]:
%%time
read_directions_part2('data/input_22.txt')

CPU times: user 306 ms, sys: 2.29 ms, total: 308 ms
Wall time: 309 ms


1234650223944734